In [2]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

print("Hello to Deep Learning")

Hello to Deep Learning


### Installing some main torch library and other essential tools.
There are a number of imports for torch. (These are not included with ISLP, so must be installed separately.) All of them are used to specify sequentially-structured networks.

In [3]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError,R2Score)
from torchinfo import summary

 The package pytorch_lightning is a somewhat higher-level interface to torch that simplifies the specification and fitting of models by reducing the amount of boilerplate code needed (compared to using torch alone).

In order to reproduce results we use seed_everything(). We will also instruct torch to use deterministic algorithms where possible.

In [4]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)
#We will use several datasets shipped with torchvision for our examples
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)
# The glob() function from the glob module is used to find all files matching wildcard characters
from glob import glob
import json

Seed set to 0


In [5]:
Hitters = load_data('Hitters').dropna()
print(Hitters.shape)
n = Hitters.shape[0]

(263, 20)


We will fit two linear models (least squares and lasso) and compare their performance to that of a neural network. For this comparison we will use mean absolute error on a validation dataset. In tools like ModelSpec (from the ISLP library), the design matrix allows you to specify transformations, interaction terms, and polynomial expansions of features.

It also allows machine learning models to understand and process the input data as well handle categorical variables (using one-hot encoding).Overall, fit_transform() in ModelSpec automates the creation of a design matrix, preparing data for machine learning models by fitting and transforming it in one step​.

In [10]:
model = MS(Hitters.columns.drop('Salary'),intercept=False)
X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary'].to_numpy()
model

ModelSpec(intercept=False,
          terms=Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object'))

The to_numpy() method above converts pandas data frames or series to numpy arrays. We do this because we will need to use sklearn to fit the lasso model, and it requires this conversion. We now split the data into test and training, fixing the random state used by sklearn to do the split.

In [16]:
(X_train, 
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)
# X_train.shape
# X_test.shape
# Y_train.shape

(175,)

### Linear Models

We fit the linear model and evaluate the test error directly.   

In [8]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs(Yhat_test - Y_test).mean()

259.71528833146294

Next we fit the lasso using sklearn. We are using mean absolute error to select and evaluate a model, rather than mean squared error. 
Pipeline is a scikit-learn class used to chain multiple steps together. In this case, it chains standardization and Lasso regression into a single workflow.

In [19]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000)
standard_lasso = Pipeline(steps=[('scaler', scaler),
                                 ('lasso', lasso)])

X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]
lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean())).max() / n
param_grid = {'lasso__alpha': np.exp(np.linspace(0, np.log(0.01), 100))
             * lam_max}
# param_grid
# lam_max

255.6575502649126

lam_max is the value of the regularization parameter (lambda) beyond which all coefficients in the Lasso regression model would be shrunk to zero. It's essentially the highest level of regularization that would still allow some coefficients to remain non-zero.

We also now perform cross validation using this sequence of tuning regularization parameter (lambda) values.The 'param_grid' can be used in conjunction with GridSearchCV to find the optimal value of lambda for Lasso regression:

In [22]:
cv = KFold(10,
           shuffle=True,
           random_state=1)
grid = GridSearchCV(standard_lasso,
                    param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train);
# grid

We extract the lasso model with best cross-validated mean absolute error, and evaluate its performance on X_test and Y_test, which were not used in cross-validation.

In [23]:
trained_lasso = grid.best_estimator_
Yhat_test = trained_lasso.predict(X_test)
np.fabs(Yhat_test - Y_test).mean()

235.6754837478029

### Specifying a Network: Classes and Inheritance
To fit the neural network, we first set up a model structure that describes the network. Doing so requires us to define new classes specific to the model we wish to fit. Typically this is done in pytorch by sub-classing a generic representation of a network, which is the approach we take here.

In [ ]:
class HittersModel(nn.Moule):

    def __init__(self,input_size):
        pass